<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Reinforcement Learning for Finance

**Chapter 06 &mdash; Algorithmic Trading**

&copy; Dr. Yves J. Hilpisch

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>

## Prediction Game Revisited

In [ ]:
!git clone https://github.com/tpq-classes/rl_4_finance.git
import sys
sys.path.append('rl_4_finance')


In [ ]:
import os
import warnings
warnings.simplefilter('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
import math
import random
import numpy as np
import pandas as pd
from pylab import plt, mpl

In [ ]:
plt.style.use('seaborn-v0_8')
# mpl.rcParams['figure.dpi'] = 300
# mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
np.set_printoptions(suppress=True)
%config InlineBackend.figure_format = 'svg'

In [ ]:
from finance import *

In [ ]:
finance = Finance('GLD', 'r', min_accuracy=47.5,
              n_features=4)

In [ ]:
finance.data[finance.symbol].plot(title=finance.symbol);

In [ ]:
from dqlagent import *

In [ ]:
random.seed(500)

In [ ]:
dqlagent = DQLAgent(finance.symbol, finance.feature,
                 finance.n_features, finance)

In [ ]:
%time dqlagent.learn(50)

In [ ]:
dqlagent.test(3)

In [ ]:
from simulation import Simulation

In [ ]:
random.seed(500)

In [ ]:
simulation = Simulation('SYMBOL', 'r', 4, '2024-1-1', '2026-1-1', 2 * 252,
                 min_accuracy=0.5,
                 x0=1, kappa=1, theta=0.75, sigma=0.1,
                 # x0=100, kappa=2, theta=50, sigma=0.1,
                 new=True, normalize=True)

In [ ]:
for _ in range(50):
    simulation.reset()
    simulation.data[simulation.symbol].plot(title=simulation.symbol);

In [ ]:
random.seed(500)

In [ ]:
agent = DQLAgent(simulation.symbol, simulation.feature,
                 simulation.n_features, simulation)

In [ ]:
%time agent.learn(250)

In [ ]:
agent.test(5)

In [ ]:
class ActionSpace:
    def sample(self):
        return random.randint(0, 1)

In [ ]:
class Trading:
    def __init__(self, symbol, features, window, lags,
                 start, end, periods,
                 x0=100, kappa=1, theta=100, sigma=0.2,
                 leverage=1, min_accuracy=0.5, min_performance=0.85,
                 mu=None, std=None,
                 new=True, normalize=True):
        self.symbol = symbol
        self.features = features
        self.n_features = len(features)
        self.window = window
        self.lags = lags
        self.start = start
        self.end = end
        self.periods = periods
        self.x0 = x0
        self.kappa = kappa
        self.theta = theta
        self.sigma = sigma
        self.leverage = leverage  # <1>
        self.min_accuracy = min_accuracy  # <2>
        self.min_performance = min_performance  # <3>
        self.start = start
        self.end = end
        self.mu = mu
        self.std = std
        self.new = new
        self.normalize = normalize
        self.action_space = ActionSpace()
        self._simulate_data()
        self._prepare_data()

In [ ]:
class Trading(Trading):
    def _simulate_data(self):
        index = pd.date_range(start=self.start,
                    end=self.end, periods=self.periods)
        s = [self.x0]
        dt = (index[-1] - index[0]).days / 365 / self.periods
        for t in range(1, len(index)):
            s_ = (s[t - 1] + self.kappa * (self.theta - s[t - 1]) * dt +
                  s[t - 1] * self.sigma * math.sqrt(dt) * random.gauss(0, 1))
            s.append(s_)
        self.data = pd.DataFrame(s, columns=[self.symbol], index=index)

In [ ]:
class Trading(Trading):
    def _prepare_data(self):
        self.data['r'] = np.log(self.data / self.data.shift(1))
        self.data.dropna(inplace=True)
        # additional features
        if self.window > 0:
            self.data['SMA'] = self.data[
                self.symbol].rolling(self.window).mean()  # <1>
            self.data['DEL'] = self.data[
                self.symbol] - self.data['SMA']  # <2>
            self.data['MIN'] = self.data[
                self.symbol].rolling(self.window).min()  # <3>
            self.data['MAX'] = self.data[
                self.symbol].rolling(self.window).max()  # <4>
            self.data['MOM'] = self.data['r'].rolling(
                self.window).mean()  # <5>
            # add additional features here
            self.data.dropna(inplace=True)
        if self.normalize:
            if self.mu is None or self.std is None:
                self.mu = self.data.mean()
                self.std = self.data.std()
            self.data_ = (self.data - self.mu) / self.std
        else:
            self.data_ = self.data.copy()
        self.data['d'] = np.where(self.data['r'] > 0, 1, 0)
        self.data['d'] = self.data['d'].astype(int)

In [ ]:
class Trading(Trading):
    def _get_state(self):
        return self.data_[self.features].iloc[self.bar -
                                self.lags:self.bar]
    def seed(self, seed):
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_random_seed(seed)
    def reset(self):
        if self.new:
            self._simulate_data()
            self._prepare_data()
        self.treward = 0
        self.accuracy = 0
        self.actions = list()
        self.returns = list()
        self.performance = 1
        self.bar = self.lags
        state = self._get_state()
        return state.values, {}

In [ ]:
class Trading(Trading):
    def step(self, action):
        correct = action == self.data['d'].iloc[self.bar]
        ret = self.data['r'].iloc[self.bar] * self.leverage
        reward_ = 1 if correct else 0
        pl = abs(ret) if correct else -abs(ret)  # <1>
        reward = reward_
        # reward = pl
        # reward = reward_ + 10 * pl
        self.treward += reward
        self.bar += 1
        self.accuracy = self.treward / (self.bar - self.lags)
        self.performance *= math.exp(pl)  # <2>
        if self.bar >= len(self.data):
            done = True
        elif reward_ == 1:
            done = False
        elif (self.accuracy < self.min_accuracy and
              self.bar > self.lags + 15):
            done = True
        elif (self.performance < self.min_performance and
              self.bar > self.lags + 15):  # <3>
            done = True
        else:
            done = False
        state = self._get_state()
        return state.values, reward, done, False, {}

In [ ]:
symbol = 'SYMBOL'

In [ ]:
trading = Trading(symbol, [symbol, 'r', 'DEL'], window=10, lags=5,
            start='2024-1-1', end='2026-1-1', periods=504,
            x0=100, kappa=2, theta=300, sigma=0.1, normalize=False)

In [ ]:
random.seed(750)

In [ ]:
trading.reset()  # <1>

In [ ]:
trading.data.info()

In [ ]:
trading.data.iloc[-200:][
    [trading.symbol, 'SMA', 'MIN', 'MAX']].plot(
        style=['-', '.', '-.', '-.'], lw=1.0);

In [ ]:
class TradingAgent(DQLAgent):
    def _create_model(self, hu, lr):
        self.model = Sequential()
        self.model.add(Dense(hu, input_dim=
            self.env.lags * self.env.n_features,
                        activation='relu'))  # <1>
        self.model.add(Flatten())  # <2>
        self.model.add(Dense(hu, activation='relu'))
        self.model.add(Dense(2, activation='linear'))
        self.model.compile(loss='mse',
            optimizer=opt(learning_rate=lr))

In [ ]:
random.seed(100)
tf.random.set_seed(100)

In [ ]:
trading = Trading(symbol, ['r', 'DEL', 'MOM'], window=10, lags=8,
            start='2024-1-1', end='2026-1-1', periods=2 * 252,
            x0=100, kappa=2, theta=50, sigma=0.1,
            leverage=1, min_accuracy=0.5, min_performance=0.85,
            new=True, normalize=True)

In [ ]:
trading.data_.iloc[-200:][
    [trading.symbol, 'SMA', 'MIN', 'MAX']].plot(
        style=['-', '.', '-.', '-.'], lw=1.0);  # normalized data

In [ ]:
tradingagent = TradingAgent(trading.symbol, trading.features,
                 trading.n_features, trading, hu=24, lr=0.0001)

In [ ]:
%%time
tradingagent.test(50, min_accuracy=0.0,
           min_performance=0.0,
           verbose=False)

In [ ]:
random_performances = tradingagent.performances  # <1>

In [ ]:
sum(random_performances) / len(random_performances)  # <2>

In [ ]:
plt.hist(random_performances, bins=30)
plt.xlabel('gross performance')
plt.ylabel('frequency');

In [ ]:
%time tradingagent.learn(500)

In [ ]:
%%time
tradingagent.test(50, min_accuracy=0.0,
           min_performance=0.0,
           verbose=False)

In [ ]:
sum(tradingagent.performances) / len(tradingagent.performances)

In [ ]:
plt.hist(random_performances, bins=30, label='random')
plt.hist(tradingagent.performances, bins=30, label='trained')
plt.xlabel('gross performance')
plt.ylabel('frequency')
plt.legend();

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>